In [1]:
import numpy as np
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer
import math

from tensorflow.keras.layers import Dense, Input, Flatten, Conv1D, MaxPool1D, BatchNormalization
from tensorflow.keras.models import Model
import tensorflow as tf
from sklearn.utils import shuffle

from tqdm import trange

##### Verify TensorFlow and Python version

In [3]:
tf.__version__, 

('2.10.0',)

In [4]:
from platform import python_version

print(python_version())

3.8.12


##### Load and process the entire dataset

In [4]:
# Define the working path
WORKING_PATH = r"/lscratch/yy10099/Virus_Mixture_LOD/Results/"

In [ ]:
# Use chunk to read a very large dataset.
chunk_size = 100000  # Adjust the size as needed
chunks = pd.read_csv(WORKING_PATH + "combined_dataframe.csv", chunksize=chunk_size)

Mix_dat = None
i=1 # i should stop at 9
for chunk in chunks:
    print(i)
    # Process each chunk
    Mix_dat = pd.concat([Mix_dat, chunk], axis=0)
    i+=1

Mix_dat=Mix_dat.drop('Unnamed: 0',axis=1)
Mix_dat

In [8]:
labels = np.unique(Mix_dat['Label'])
concs = np.unique(Mix_dat['Conc'])
print(labels, concs)

["['Ad5']" "['CoV229E']" "['CoVNL63', 'FluB']"
 "['CoVNL63', 'H1N1', 'RSVA2']" "['CoVNL63', 'H1N1', 'RSVB1']"
 "['CoVNL63', 'H1N1']" "['CoVNL63', 'H3N2', 'RSVA2']"
 "['CoVNL63', 'H3N2', 'RSVB1']" "['CoVNL63', 'H3N2']"
 "['CoVNL63', 'RSVA2']" "['CoVNL63', 'RSVB1']" "['CoVNL63']" "['CoVOC43']"
 "['FluB']" "['H1N1', 'RSVA2']" "['H1N1', 'RSVB1']" "['H1N1']"
 "['H3N2', 'RSVA2']" "['H3N2', 'RSVB1']" "['H3N2']" "['HMPVA']"
 "['HMPVB']" "['RSVA2']" "['RSVB1']" "['saliva']"] ['[10.0]' '[100.0, 100.0]' '[100.0, 100000.0]' '[100.0, 12500.0]'
 '[100.0, 1562.0]' '[100.0, 195.0]' '[100.0, 25000.0]' '[100.0, 3125.0]'
 '[100.0, 391.0]' '[100.0, 50.0]' '[100.0, 50000.0]' '[100.0, 6250.0]'
 '[100.0, 781.0]' '[100.0]' '[100000.0, 100.0]'
 '[100000.0, 100000.0, 100000.0]' '[100000.0, 100000.0, 1562.0]'
 '[100000.0, 100000.0, 195.0]' '[100000.0, 100000.0, 25000.0]'
 '[100000.0, 100000.0, 391.0]' '[100000.0, 100000.0, 395.0]'
 '[100000.0, 100000.0, 50000.0]' '[100000.0, 100000.0, 6250.0]'
 '[100000.0, 10000

In [60]:
print(Mix_dat.groupby(['Label','Conc'], group_keys=False).apply(lambda x: x.shape))

Label       Conc      
['Ad5']     [100.0]       (399, 1253)
            [100000.0]    (392, 1253)
            [12500.0]     (392, 1253)
            [1562.0]      (398, 1253)
            [195.0]       (399, 1253)
                             ...     
['RSVB1']   [50000.0]     (396, 1253)
            [6250.0]      (399, 1253)
            [781.0]       (395, 1253)
['saliva']  [10.0]        (356, 1253)
            [20.0]        (391, 1253)
Length: 3478, dtype: object


#### We use 100% of total data.

In [61]:
frac = 1 # We used 100% of the total data. If you use a computer with less computing power or less RAM space, you can use a small portion of the data.
subset = Mix_dat.groupby(['Label','Conc'], group_keys=False).apply(lambda x: x.sample(math.ceil(x.shape[0]*frac)))

In [62]:
print(subset.shape)
print(subset.groupby(['Label','Conc'], group_keys=False).apply(lambda x: x.shape))

(1213550, 1253)
Label       Conc      
['Ad5']     [100.0]       (399, 1253)
            [100000.0]    (392, 1253)
            [12500.0]     (392, 1253)
            [1562.0]      (398, 1253)
            [195.0]       (399, 1253)
                             ...     
['RSVB1']   [50000.0]     (396, 1253)
            [6250.0]      (399, 1253)
            [781.0]       (395, 1253)
['saliva']  [10.0]        (356, 1253)
            [20.0]        (391, 1253)
Length: 3478, dtype: object


Data splitting

In [7]:
# Split the data into train, validation, and test sets, fraction = 8:1:1, adjust if needed.
def split_data(data, train_frac=0.8, val_frac=0.1):
    tra_val = data.groupby(['Label', 'Conc']).apply(
        lambda x: x.sample(math.ceil(x.shape[0] * (train_frac + val_frac)))
    ).reset_index(drop=True)
    
    test = data.loc[data.index.difference(tra_val.index)]
    
    train = tra_val.groupby(['Label', 'Conc']).apply(
        lambda x: x.sample(math.ceil(x.shape[0] * (train_frac / (train_frac + val_frac))))
    ).reset_index(drop=True)
    
    val = tra_val.loc[tra_val.index.difference(train.index)]
    
    return train, val, test

# Perform the split
train, val, test = split_data(subset)

In [8]:
# Save the datasets.
# pd.DataFrame(train).to_csv(WORKING_PATH + r"train.csv",index=False)
# pd.DataFrame(val).to_csv(WORKING_PATH + r"validation.csv",index=False)
# pd.DataFrame(test).to_csv(WORKING_PATH + r"test.csv",index=False)

Load the data

In [5]:
val = pd.read_csv(WORKING_PATH + r"validation.csv")
val.shape

(88172, 1253)

In [6]:
test = pd.read_csv(WORKING_PATH + r"test.csv")
test.shape

(87922, 1253)

In [7]:
# train = pd.read_csv(WORKING_PATH + r"train.csv")
chunk_size = 100000  # Adjust the size as needed
chunks = pd.read_csv(WORKING_PATH + r"train.csv", chunksize=chunk_size)

train = None
i=1 # i should stop at 8
for chunk in chunks:
    print(i)
    # Process each chunk
    train = pd.concat([train, chunk], axis=0)
    i+=1
train.shape

1
2
3
4
5
6
7
8


(714436, 1253)

In [ ]:
def prepare_features(data):
    """Prepare features (X) from the input data."""
    return data.iloc[:, :-2].values

def reshape_features(X):
    """Reshape features for model input."""
    sample_size = X.shape[0]
    time_steps = X.shape[1]
    input_dimension = 1
    return X.reshape(sample_size, time_steps, input_dimension)

def convert_labels_to_list(data):
    """Convert string labels and concentrations to lists."""
    labels = data['Label'].values
    concs = data['Conc'].values
    
    label_list = []
    conc_list = []
    for i in range(data.shape[0]):
        label_list.append(ast.literal_eval(labels[i]))
        conc_list.append(ast.literal_eval(concs[i]))
    
    return label_list, conc_list

def prepare_classification_labels(labels):
    """Prepare classification labels using MultiLabelBinarizer."""
    mlb = MultiLabelBinarizer()
    y_class = mlb.fit_transform(labels)
    return pd.DataFrame(y_class, columns=mlb.classes_), mlb

def prepare_regression_labels(y_class, conc_list):
    """Prepare regression labels."""
    y_reg = np.zeros(y_class.shape)
    for i in range(y_class.shape[0]):
        y_reg[i][(y_class.values)[i]==1] = np.log10([x + 1e-20 for x in conc_list[i]])
    return pd.DataFrame(y_reg, columns=y_class.columns)

# Prepare features
X_train = prepare_features(train)
X_test = prepare_features(test)
X_val = prepare_features(val)

# Get sample sizes and time steps
sample_size_train, time_steps_train = X_train.shape
sample_size_test, time_steps_test = X_test.shape
sample_size_val, time_steps_val = X_val.shape

print(f"Training samples: {sample_size_train}")
print(f"Time steps: {time_steps_train}")

# Reshape features
X_train_reshape = reshape_features(X_train)
X_test_reshape = reshape_features(X_test)
X_val_reshape = reshape_features(X_val)

# Convert labels to lists
label_train, conc_train = convert_labels_to_list(train)
label_test, conc_test = convert_labels_to_list(test)
label_val, conc_val = convert_labels_to_list(val)

# Prepare classification labels
y_train_class, mlb1 = prepare_classification_labels(label_train)
y_test_class, mlb2 = prepare_classification_labels(label_test)
y_val_class, mlb3 = prepare_classification_labels(label_val)

# Prepare regression labels
y_train_reg = prepare_regression_labels(y_train_class, conc_train)
y_test_reg = prepare_regression_labels(y_test_class, conc_test)
y_val_reg = prepare_regression_labels(y_val_class, conc_val)

print("Data preprocessing completed.")
print(f"X_train_reshape shape: {X_train_reshape.shape}")
print(f"y_train_class shape: {y_train_class.shape}")
print(f"y_train_reg shape: {y_train_reg.shape}")

#### Multi-task model training

In [ ]:
# Model definition
def MixNet_10_Conv(n_timesteps, n_inputs, n_outputs, n_layers):
    # Input layer
    visible = Input(shape=(n_timesteps, n_inputs,))
    
    # Convolutional blocks
    filters = 64  # Starting number of filters
    hidden = visible
    for i in range(n_layers):
        hidden = Conv1D(filters=filters, kernel_size=3, activation='relu', padding='same', name=f'Conv1D_{i+1}')(hidden)
        hidden = BatchNormalization(name=f"BatchNorm_{i+1}")(hidden)
        if i < n_layers - 1: # Add pooling except for the last convolutional layer
            hidden = MaxPool1D(pool_size=2, strides=2, name=f'MaxPooling1D_{i+1}')(hidden)
        if i < 7: # Increase the number of filters for the first 8 blocks
            filters = min(filters * 2, 512) # Limit the maximum number of filters

    hidden = Flatten()(hidden)
    hidden = Dense(200, activation='relu', name='Dense_1')(hidden)
    hidden = Dense(100, activation='relu', name='Dense_2')(hidden)

    out_reg = Dense(n_outputs, activation='linear', name="Reg")(hidden)
    out_clas = Dense(n_outputs, activation='sigmoid', name="Class")(hidden)

    model = Model(inputs=visible, outputs=[out_clas, out_reg], name="MixNet_10_Conv")
    model.compile(optimizer='adam', 
                  loss={'Class': 'binary_crossentropy', 'Reg': 'mae'},
                  metrics=['acc', 'mse'],
                  loss_weights={'Class': 1, 'Reg': 1})


In [18]:
# Model initialization
model = MixNet_10_Conv(n_timesteps=1251, n_inputs=1, n_outputs=12, n_layers=10)
model.summary()

2024-08-29 15:49:24.371152: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 15:49:30.808830: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79084 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:81:00.0, compute capability: 8.0


Model: "MixNet_10_Conv"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1251, 1)]    0           []                               
                                                                                                  
 Conv1D_1 (Conv1D)              (None, 1251, 64)     256         ['input_1[0][0]']                
                                                                                                  
 BatchNorm_1 (BatchNormalizatio  (None, 1251, 64)    256         ['Conv1D_1[0][0]']               
 n)                                                                                               
                                                                                                  
 MaxPooling1D_1 (MaxPooling1D)  (None, 625, 64)      0           ['BatchNorm_1[0][0]'

In [20]:
# Define custom loss functions
def calculate_cross_entropy_matrix(y_true_matrix, y_pred_matrix):
    # Convert y_true_matrix to a TensorFlow tensor if it's a pandas DataFrame or Series
    if isinstance(y_true_matrix, (pd.DataFrame, pd.Series)):
        y_true_matrix = tf.convert_to_tensor(y_true_matrix.values, dtype=tf.float32)
    else:
        y_true_matrix = tf.convert_to_tensor(y_true_matrix, dtype=tf.float32)
#     print(y_true_matrix)

    # Ensure y_pred_matrix is a TensorFlow tensor
    y_pred_matrix = tf.convert_to_tensor(y_pred_matrix, dtype=tf.float32)

    # Clip the prediction values to prevent log(0) which can result in -inf
    y_pred_matrix = tf.clip_by_value(y_pred_matrix, 1e-15, 1 - 1e-15)
#     print(y_pred_matrix)
    
    # Similarly, clip (1 - y_pred_matrix) to prevent log(0) when y_pred_matrix is 1
    one_minus_y_pred_matrix = tf.clip_by_value(1 - y_pred_matrix, 1e-15, 1 - 1e-15)

    # Calculate cross-entropy
    cross_entropy = -tf.reduce_sum(y_true_matrix * tf.math.log(y_pred_matrix) + 
                                   (1 - y_true_matrix) * tf.math.log(one_minus_y_pred_matrix), axis=1)
    return tf.reduce_mean(cross_entropy)


def custom_MAE(y_true_reg, y_pred_reg, y_true_class):
    # Ensure all inputs are of the same type, preferably float32
    y_true_reg = tf.cast(y_true_reg, dtype=tf.float32)
    y_pred_reg = tf.cast(y_pred_reg, dtype=tf.float32)
    y_true_class = tf.cast(y_true_class, dtype=tf.float32)

    # Calculate the absolute differences only where y_true_class is 1
    error = tf.abs(y_true_reg - y_pred_reg) * y_true_class

    # Sum the errors and count the number of non-zero entries in y_true_class for normalization
    sum_error = tf.reduce_sum(error)
    num_non_zero = tf.reduce_sum(y_true_class)

    # Avoid division by zero
    return sum_error / (num_non_zero + tf.keras.backend.epsilon())
    
def calculate_instance_based_metrics(y_true, y_pred):
    # Ensure both y_true and y_pred are TensorFlow tensors
    y_true = tf.convert_to_tensor(y_true, dtype=tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, dtype=tf.float32)

    # Convert predictions to binary using a threshold (e.g., 0.5)
    y_pred_binary = tf.cast(y_pred > 0.5, tf.float32)

    # Calculate instance-based metrics
    correct_predictions = tf.cast(tf.reduce_all(tf.equal(y_true, y_pred_binary), axis=1), tf.float32)
    accuracy = tf.reduce_mean(correct_predictions)

    # Precision, recall, and F1 for each instance
    true_positives = tf.reduce_sum(y_true * y_pred_binary, axis=1)
    predicted_positives = tf.reduce_sum(y_pred_binary, axis=1)
    actual_positives = tf.reduce_sum(y_true, axis=1)

    # Avoid division by zero
    epsilon = tf.keras.backend.epsilon()

    precision = tf.reduce_sum(true_positives) / (tf.reduce_sum(predicted_positives) + epsilon)
    recall = tf.reduce_sum(true_positives) / (tf.reduce_sum(actual_positives) + epsilon)
    f1 = 2 * precision * recall / (precision + recall + epsilon)

    return accuracy.numpy(), precision.numpy(), recall.numpy(), f1.numpy()

In [ ]:
# Training parameters, change if needed
SAVE_PATH = "/scratch/yy10099/Virus_Mixture_LOD/Results/Model5"
optimizer = tf.keras.optimizers.Adam(lr=0.001)
total_epochs = 500
epoch_save_once = 100
batch_size = 1024 # 1024, 2048, 4096 will have similar speed, reduce RAM by using 1024
classification_weight = 1
regression_weight = 1000

In [ ]:
# Initialize lists to store metrics
acc_train, entropy_train, original_MAE_train, custom_MAE_train, f1_train, precision_train, recall_train = [], [], [], [], [], [], []
acc_val, entropy_val, original_MAE_val, custom_MAE_val, f1_val, precision_val, recall_val = [], [], [], [], [], [], []

# Custom training loop
for epoch in range(total_epochs):
    print('Working on epoch:', epoch)

    # Shuffle training data
    X_train, y_class_train, y_reg_train = shuffle(X_train_reshape, y_train_class, y_train_reg)

    # Training loop
    for batch in trange(0, len(X_train), batch_size):
#         print("batch: " + str(batch) + "/" + str(np.ceil(len(X_train)/batch_size)))
        x_batch_train = X_train[batch:batch+batch_size]
        y_batch_class = y_class_train[batch:batch+batch_size]
        y_batch_reg = y_reg_train[batch:batch+batch_size]

        with tf.GradientTape() as tape:
            y_pred_class, y_pred_reg = model(x_batch_train, training=True)
            # print(y_batch_class)
            # print(y_pred_class)

            # Now using the modified TensorFlow-based cross-entropy function
            loss_class = calculate_cross_entropy_matrix(y_batch_class, y_pred_class)
            # train_acc = accuracy_score(y_batch_class, y_pred_class.numpy().round())
            # f1_train = f1_score(y_batch_class, y_pred_class.numpy().round(), average='micro')
            # precision_train = precision_score(y_batch_class, y_pred_class.numpy().round(), average='micro')
            # recall_train = recall_score(y_batch_class, y_pred_class.numpy().round(), average='micro')
            # print("Accuracy_batch: " + str(train_acc)+ ", F1: " + str(f1_train) + ", precision: " + str(precision_train) + ", recall: " + str(recall_train))
            loss_reg = custom_MAE(y_batch_reg, y_pred_reg, y_batch_class)
            total_loss = classification_weight * loss_class + regression_weight * loss_reg
                    
        # Compute gradients and update model weights
        grads = tape.gradient(total_loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        # print("Classification loss: " + str(loss_class.numpy())+ ", Regression loss: " + str(loss_reg.numpy()) + ", Total_loss: " + str(total_loss.numpy()))

    # ---------------------------Post-epoch: calculate metrics for training data---------------------------
    y_train_pred_class, y_train_pred_reg = model.predict(X_train)
    y_train_pred_class_rounded = y_train_pred_class.round()

    # train_acc = accuracy_score(y_class_train, y_train_pred_class_rounded)
    train_cross_entropy = calculate_cross_entropy_matrix(y_class_train, y_train_pred_class).numpy()
    # train_f1 = f1_score(y_class_train, y_train_pred_class_rounded, average='micro')
    # train_precision = precision_score(y_class_train, y_train_pred_class_rounded, average='micro')
    # train_recall = recall_score(y_class_train, y_train_pred_class_rounded, average='micro')

    # Example usage:
    train_acc, train_precision, train_recall, train_f1 = calculate_instance_based_metrics(y_class_train.to_numpy(), y_train_pred_class_rounded)

    acc_train.append(train_acc)
    entropy_train.append(train_cross_entropy)
    f1_train.append(train_f1)
    precision_train.append(train_precision)
    recall_train.append(train_recall)
    
    # Calculate original MAE (all viruses)
    original_train_mae = (y_reg_train - y_train_pred_reg).abs().mean().mean()
    original_MAE_train.append(original_train_mae)

    # Calculate custom MAE (only present viruses)
    custom_train_mae = custom_MAE(y_reg_train, y_train_pred_reg, y_class_train).numpy()
    custom_MAE_train.append(custom_train_mae)
    
    print("Training accuracy: " + str(train_acc)+ ", Training F1 score: " + str(train_f1) + ", Training Precision: " + str(train_precision) + ", Training Recall: " + str(train_recall))
    print("Training cross entropy: " + str(train_cross_entropy) + ", Training MAE consider 0: " + str(original_train_mae) + ", Training MAE do not consider 0: " + str(custom_train_mae))
    
    # ---------------------------Validation---------------------------
    # Validation at the end of epoch
    y_val_pred_class, y_val_pred_reg = model.predict(X_val_reshape)
    y_val_pred_class_rounded = y_val_pred_class.round()

    # Calculate and append metrics
    # val_acc = accuracy_score(y_val_class, y_val_pred_class_rounded)
    val_cross_entropy = calculate_cross_entropy_matrix(y_val_class, y_val_pred_class).numpy()
    # val_f1 = f1_score(y_val_class, y_val_pred_class_rounded, average='micro')
    # val_precision = precision_score(y_val_class, y_val_pred_class_rounded, average='micro')
    # val_recall = recall_score(y_val_class, y_val_pred_class_rounded, average='micro')
    val_acc, val_precision, val_recall, val_f1 = calculate_instance_based_metrics(y_val_class, y_val_pred_class_rounded)

    acc_val.append(val_acc)
    entropy_val.append(val_cross_entropy)
    f1_val.append(val_f1)
    precision_val.append(val_precision)
    recall_val.append(val_recall)
    
    # Calculate original MAE for validation data (all viruses)
    original_val_mae = (y_val_reg - y_val_pred_reg).abs().mean().mean()
    original_MAE_val.append(original_val_mae)

    # Calculate custom MAE for validation data (only present viruses)
    custom_val_mae = custom_MAE(y_val_reg, y_val_pred_reg, y_val_class).numpy()
    custom_MAE_val.append(custom_val_mae)

    print("Validation accuracy: " + str(val_acc)+ ", Validation F1 score: " + str(val_f1) + ", Validation Precision: " + str(val_precision) + ", Validation Recall: " + str(val_recall))
    print("Validation cross entropy: " + str(val_cross_entropy) + ", Validation MAE consider 0: " + str(original_val_mae) + ", Validation MAE do not consider 0: " + str(custom_val_mae))
    
    # ---------------------------Save model and metrics---------------------------
    if epoch % epoch_save_once == 0 or epoch == total_epochs - 1:
        model.save(f'{SAVE_PATH}/Subset_Model_{epoch}.h5')
        metrics_df = pd.DataFrame({
            'Epoch': range(1, epoch + 2),
            'Train_Accuracy': acc_train,
            'Train_Cross_Entropy': entropy_train,
            'Train_Original_MAE': original_MAE_train,# "Original" means we consider every position, including 0 and 1
            'Train_Custom_MAE': custom_MAE_train, # "Custom" means we only consider the position with virus (1)
            'Train_F1_Score': f1_train,
            'Train_Precision': precision_train,
            'Train_Recall': recall_train,
            'Validation_Accuracy': acc_val,
            'Validation_Cross_Entropy': entropy_val,
            'Validation_Original_MAE': original_MAE_val,
            'Validation_Custom_MAE': custom_MAE_val,
            'Validation_F1_Score': f1_val,
            'Validation_Precision': precision_val,
            'Validation_Recall': recall_val
        })
        metrics_df.to_csv(f'{SAVE_PATH}/Recursive_Model_Summary_{epoch}.csv', index=False)

    print('Epoch completed:', epoch)


Working on epoch: 0


  0%|                                                   | 0/698 [00:00<?, ?it/s]2024-08-29 15:50:10.914478: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2024-08-29 15:50:20.046265: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-08-29 15:50:25.618454: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0xed949b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-08-29 15:50:25.618494: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2024-08-29 15:50:26.094458: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-08-29 15:50:27.056108: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled

100%|█████████████████████████████████████████| 698/698 [02:04<00:00,  5.60it/s]


22327/22327 [==============================] - 44s 2ms/step
Training accuracy: 0.1749002, Training F1 score: 0.71558654, Training Precision: 0.7747372, Training Recall: 0.6648275
Training cross entropy: 2.799267, Training MAE consider 0: 2.381665682516608, Training MAE do not consider 0: 0.16943641
2756/2756 [==============================] - 5s 2ms/step
Validation accuracy: 0.17426167, Validation F1 score: 0.7152279, Validation Precision: 0.7743405, Validation Recall: 0.6645004
Validation cross entropy: 2.7997344, Validation MAE consider 0: 2.381986929151217, Validation MAE do not consider 0: 0.16978045
Epoch completed: 0
Working on epoch: 1


100%|█████████████████████████████████████████| 698/698 [01:40<00:00,  6.93it/s]


...

100%|█████████████████████████████████████████| 698/698 [01:38<00:00,  7.05it/s]


22327/22327 [==============================] - 43s 2ms/step
Training accuracy: 0.9947651, Training F1 score: 0.99844307, Training Precision: 0.99854803, Training Recall: 0.9983383
Training cross entropy: 0.03092318, Training MAE consider 0: 2.6148990835125017, Training MAE do not consider 0: 0.008485174
2756/2756 [==============================] - 5s 2ms/step
Validation accuracy: 0.9912444, Validation F1 score: 0.9972525, Validation Precision: 0.997388, Validation Recall: 0.9971172
Validation cross entropy: 0.062882625, Validation MAE consider 0: 2.6171911693068814, Validation MAE do not consider 0: 0.020589478
Epoch completed: 431
Working on epoch: 432


100%|█████████████████████████████████████████| 698/698 [01:39<00:00,  7.04it/s]


22327/22327 [==============================] - 43s 2ms/step
Training accuracy: 0.99516404, Training F1 score: 0.99847627, Training Precision: 0.9984133, Training Recall: 0.9985394
Training cross entropy: 0.031845175, Training MAE consider 0: 2.6174943849658057, Training MAE do not consider 0: 0.017135406
2756/2756 [==============================] - 5s 2ms/step
Validation accuracy: 0.9915733, Validation F1 score: 0.99730074, Validation Precision: 0.99719024, Validation Recall: 0.9974114
Validation cross entropy: 0.061649933, Validation MAE consider 0: 2.6179633861561658, Validation MAE do not consider 0: 0.029138824
Epoch completed: 432
Working on epoch: 433


  5%|██▎                                       | 38/698 [00:05<01:35,  6.93it/s]